### Week 2 - Adding Sales Report - Exercise 1 Task 1

In [ ]:
USE `LittleLemonDB`;
CREATE  OR REPLACE VIEW `OrdersView` AS SELECT OrderID, Quantity, TotalCost FROM Orders WHERE Quantity > 2;

### Week 2 - Adding Sales Report - Exercise 1 Task 2

In [ ]:
SELECT Customers.CustomerID, Customers.CustomerName, Orders.OrderID, Orders.TotalCost, Menus.MenuName, MenuItems.CourseName, MenuItems.StarterName FROM Customers
INNER JOIN Orders ON Customers.CustomerID = Orders.CustomerID
INNER JOIN Menus ON Orders.MenuID = Menus.MenuID
INNER JOIN MenuItems ON Menus.MenuItemID = MenuItems.MenuItemID
WHERE TotalCost > 150

### Week 2 - Adding Sales Report - Exercise 1 Task 3

In [ ]:
Select MenuName FROM Menus WHERE MenuID IN (SELECT MenuID FROM Orders WHERE Quantity > 2)

### Week 2 - Adding Sales Report - Exercise 2 Task 1

In [ ]:
CREATE PROCEDURE `GetMaxQuantity`() SELECT MAX(Quantity) AS "Max Quantity in Order" FROM Orders

### Week 2 - Adding Sales Report - Exercise 2 Task 2

In [ ]:
PREPARE GetOrderDetail FROM "SELECT OrderID, Quantity, TotalCost FROM Orders WHERE CustomerID = ?";

SET @id = 1;
EXECUTE GetOrderDetail USING @id;

### Week 2 - Adding Sales Report - Exercise 2 Task 3

In [ ]:
DROP PROCEDURE IF EXISTS CancelOrder;
DELIMITER //
CREATE PROCEDURE CancelOrder(IN order_id INT)
BEGIN
DELETE FROM Orders WHERE OrderId = order_id;
SELECT CONCAT("Order ", order_id, " is cancelled") AS Confirmation;
END //
DELIMITER ;